# <center> <font color='green'> <b> Functional Programming </b> </font> </center>


## Table of contents
- [1 - Definition](#1)
- [2 - Iterators](#2)
- [3 - Generators and list comprehensions](#3)
- [4 - Built-in functions](#4)
    - [4.1 - Map](#4.1)
    - [4.2 - Filter](#4.2)
- [5 - Functools](#5)
    - [5.1 - Reduce](#5.1)
- [6 - Lambda Functions](#6)



<a name="1"></a>
## <b> <font color=' #16a085'> 1. Definition  </b> </font>

In computer science, functional programming is a programming paradigm where programs are constructed by applying and composing functions. It is a declarative programming paradigm in which function definitions are trees of expressions that map values to other values, rather than a sequence of imperative statements which update the running state of the program.

In functional programming, functions are treated as first-class citizens, meaning that they can be bound to names (including local identifiers), passed as arguments, and returned from other functions, just as any other data type can. This allows programs to be written in a declarative and composable style, where small functions are combined in a modular manner.


https://en.wikipedia.org/wiki/Functional_programming

Python is not a functional programming language, but it provides tools typical of functional languages.

Functional programming decomposes a problem into a set of functions. Ideally, functions only take inputs and produce outputs, and don’t have any internal state that affects the output produced for a given input. Well-known functional languages include the ML family (Standard ML, OCaml, and other variants) and Haskell.

https://docs.python.org/3.8/howto/functional.html


<a name="2"></a>
## <b> <font color=' #16a085'> 2. Iterators </b> </font>

An iterator is an object representing a stream of data; this object returns the data one element at a time. A Python iterator must support a method called __next__() that takes no arguments and always returns the next element of the stream. If there are no more elements in the stream, __next__() must raise the StopIteration exception. Iterators don’t have to be finite, though; it’s perfectly reasonable to write an iterator that produces an infinite stream of data.

The built-in iter() function takes an arbitrary object and tries to return an iterator that will return the object’s contents or elements, raising TypeError if the object doesn’t support iteration. Several of Python’s built-in data types support iteration, the most common being lists and dictionaries. An object is called iterable if you can get an iterator for it.

In [8]:
L = [1, 2, 3]

it = iter(L)

it

In [9]:
it.__next__()

1

In [10]:
next(it) # equivalent to __next__()

2

In [11]:
next(it)

3

In [12]:
next(it)

StopIteration: 

Note that you can only go forward in an iterator; there’s no way to get the previous element, reset the iterator, or make a copy of it. Iterator objects can optionally provide these additional capabilities, but the iterator protocol only specifies the __next__() method. Functions may therefore consume all of the iterator’s output, and if you need to do something different with the same stream, you’ll have to create a new iterator.

<a name="3"></a>
## <b> <font color=' #16a085'> 3. Generators and list comprehensions </b> </font>


Two common operations on an iterator’s output are:

- Performing some operation for every element.
- Selecting a subset of elements that meet some condition. 
    - For example, given a list of strings, you might want to strip off trailing whitespace from each line or extract all the strings containing a given substring.

List comprehensions and generator expressions are a concise notation for such operations. 



In [17]:
# strip all whitespace from a stream of strings

line_list = ['  line 1  ', 'line 2  ']

# Generator expression -- returns iterator
stripped_iter = (line.strip() for line in line_list)

# List comprehension -- returns list
stripped_list = [line.strip() for line in line_list]


print('list',line_list)
print('stripped',stripped_list)


list ['  line 1  ', 'line 2  ']
stripped ['line 1', 'line 2']


With a list comprehension, you get back a Python list; stripped_list is a list containing the resulting lines, not an iterator. Generator expressions return an iterator that computes the values as necessary, not needing to materialize all the values at once. This means that list comprehensions aren’t useful if you’re working with iterators that return an infinite stream or a very large amount of data. Generator expressions are preferable in these situations.

Generator expressions are surrounded by parentheses (“()”) and list comprehensions are surrounded by square brackets (“[]”). 

Generators are a special class of functions that simplify the task of writing iterators. Regular functions compute a value and return it, but generators return an iterator that returns a stream of values.

You’re doubtless familiar with how regular function calls work in Python or C. When you call a function, it gets a private namespace where its local variables are created. When the function reaches a return statement, the local variables are destroyed and the value is returned to the caller. A later call to the same function creates a new private namespace and a fresh set of local variables. But, what if the local variables weren’t thrown away on exiting a function? What if you could later resume the function where it left off? This is what generators provide; they can be thought of as resumable functions.

In [18]:
# example of generator function

def generate_ints(N):
   for i in range(N):
       yield i

In [21]:
g = generate_ints(3)
g

<generator object generate_ints at 0x7f6900429690>

In [23]:
next(g)

0

In [24]:
next(g)

1

Any function containing a yield keyword is a generator function.

When you call a generator function, it doesn’t return a single value; instead it returns a generator object that supports the iterator protocol. On executing the yield expression, the generator outputs the value of i, similar to a return statement. The big difference between yield and a return statement is that on reaching a yield the generator’s state of execution is suspended and local variables are preserved. On the next call to the generator’s __next__() method, the function will resume executing.

Let's pass values to a generator.

Values are sent into a generator by calling its send(value) method. This method resumes the generator’s code and the yield expression returns the specified value. If the regular __next__() method is called, the yield returns None.

Here’s a simple counter that increments by 2 and allows changing the value of the internal counter.

In [29]:
def counter(maximum):
    i = 0
    while i < maximum:
        val = (yield i)
        # If value provided, change counter
        if val is not None:
            i = val
        else:
            i += 2

In [30]:
it = counter(5)

In [31]:
next(it)

0

In [32]:
next(it)

2

Because yield will often be returning None, you should always check for this case. Don’t just use its value in expressions unless you’re sure that the send() method will be the only method used to resume your generator function.

Generators also become coroutines, a more generalized form of subroutines. Subroutines are entered at one point and exited at another point (the top of the function, and a return statement), but coroutines can be entered, exited, and resumed at many different points (the yield statements).

<a name="4"></a>
## <b> <font color=' #16a085'> 4. Built-in Functions </b> </font>

<a name="4.1"></a>
### 4.1. Map

The map() function applies a given function to each item of an iterable (such as a list, tuple, or set) and returns an iterator that yields the results.

The general syntax of the map() function is as follows:
    
<font color='blue'>map</font>(function, iterable1, iterable2, ...)


- function: The function to apply to each element of the iterable(s).
- iterable1, iterable2, etc.: The iterable(s) whose elements will be passed as arguments to the function.

Returns an iterator that generates the results of applying the function to the elements of the iterables one by one.

In [34]:
# Define a function to square a number
def square(x):
    return x ** 2

# Create a list of numbers
numbers = [1, 2, 3, 4, 5]

# Apply the square function to each element of the list using map()
squared_numbers = map(square, numbers)

# Convert the iterator to a list to see the results
print(list(squared_numbers))  # Output: [1, 4, 9, 16, 25]


[1, 4, 9, 16, 25]


In this example, the square() function is applied to each element of the numbers list using map(), resulting in a new iterator (squared_numbers) that yields the squared values of each element in the original list.

<a name="4.2"></a>
### 4.2. Filter

 the filter() function constructs an iterator from elements of an iterable for which a specified function returns True. It filters out elements that do not satisfy the given condition.

The general syntax of the filter() function is as follows:

<font color='blue'>filter</font>(function, iterable)

- function: The function that returns True or False based on the condition to filter elements.
- iterable: The iterable from which elements will be filtered.


Returns an iterator that yields the elements from the iterable for which the function returns True.



In [36]:
# Define a function to check if a number is even
def is_even(x):
    return x % 2 == 0

# Create a list of numbers
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Filter out the even numbers using the is_even function
even_numbers = filter(is_even, numbers)

# Convert the iterator to a list to see the results
print(list(even_numbers))  # Output: [2, 4, 6, 8, 10]


[2, 4, 6, 8, 10]


In this example, the is_even() function is applied to each element of the numbers list using filter(). The function filters out elements that are not even, resulting in a new iterator (even_numbers) that yields only the even numbers from the original list.

<a name="5"></a>
## <b> <font color=' #16a085'> 5. Functools </b> </font>

<a name="5.1"></a>
### 5.1. Reduce

We can use reduce to reduce all elements of the input to a single value by applying a specific criterion.

The general syntax of the reduce() function is as follows:

<font color='blue'>reduce</font>(function, iterable[, initializer])

- function: The function that performs the computation.
- iterable: The iterable sequence of elements to apply the function to.
- initializer (optional): The initial value that will be used as the first argument to the first call of the function. If not provided, the first item in the iterable will be used.


In [38]:
from functools import reduce

# Define a function to calculate the product of two numbers
def multiply(x, y):
    return x * y

# Create a list of numbers
numbers = [1, 2, 3, 4, 5]

# Use reduce() to calculate the product of all numbers in the list
product = reduce(multiply, numbers)

print(product)  # Output: 120 (1*2*3*4*5 = 120)


120


In this example, the multiply() function is applied cumulatively to the elements of the numbers list using reduce(), resulting in the product of all the numbers in the list.

<a name="6"></a>
## <b> <font color=' #16a085'> 6. Lambda Functions </b> </font>

A lambda function in Python is a small anonymous function defined using the lambda keyword. Lambda functions can have any number of parameters, but they can only have one expression. They are useful for writing short, throwaway functions without the need to explicitly name them.

Here's a basic syntax of a lambda function:

   <font color='blue'>lambda</font> arguments: expression 

- lambda: The keyword used to define a lambda function.
- arguments: The parameters or arguments of the function.
- expression: The single expression evaluated and returned by the function.

Lambda functions are often used in situations where you need a simple function for a short period and don't want to go through the process of defining a formal function using the def keyword.


In [40]:
square = lambda x: x ** 2
print(square(5))  # Output: 25

25


In this example, lambda x: x ** 2 defines a lambda function that takes a single argument x and returns its square. The square variable then holds this lambda function, and we call it with square(5) to get the square of 5.

In [43]:
another = lambda x,y: (x+y)/2
another(2,3) # (2+3)/2


2.5

In [45]:
# even shorter
(lambda a, b: a + b)(2, 4) # 2+4

6

In [46]:
# return more than 1 output
x = lambda a, b: (b, a)
print(x(3, 9)) # (9,3)

(9, 3)
